In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics.functional as metrics
from torch.utils.data import Dataset, DataLoader, random_split
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import ImageFolder


In [2]:
image_path = '../../Data/albumart/'

image = '3824.jpg'

image_path+image

cv2.imread(image_path+image)

[ WARN:0@0.027] global loadsave.cpp:248 findDecoder imread_('../../Data/albumart/3824.jpg'): can't open/read file: check file path/integrity


In [3]:
trans = transforms.Compose([transforms.Resize((25,25)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                    ])

In [4]:
datasets = ImageFolder(root=image_path, transform = trans)
datasets

Dataset ImageFolder
    Number of datapoints: 366
    Root location: ../../Data/albumart/
    StandardTransform
Transform: Compose(
               Resize(size=(25, 25), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [5]:
datasets.classes, datasets.class_to_idx, datasets.imgs[:5]

(['1990', '2000', '2010'],
 {'1990': 0, '2000': 1, '2010': 2},
 [('../../Data/albumart/1990/10291.jpg', 0),
  ('../../Data/albumart/1990/104099.jpg', 0),
  ('../../Data/albumart/1990/104184.jpg', 0),
  ('../../Data/albumart/1990/10435.jpg', 0),
  ('../../Data/albumart/1990/104673.jpg', 0)])

In [6]:
datasets.__getitem__(0)

(tensor([[[-0.7647, -0.7255, -0.7098,  ..., -0.9216, -0.9451, -0.9608],
          [-0.6941, -0.6706, -0.6627,  ..., -0.8902, -0.9137, -0.9373],
          [-0.6471, -0.6235, -0.6157,  ..., -0.8588, -0.8824, -0.9137],
          ...,
          [-0.7569, -0.7255, -0.7333,  ..., -0.8275, -0.8196, -0.8431],
          [-0.8118, -0.8039, -0.7961,  ..., -0.8431, -0.8510, -0.8745],
          [-0.8510, -0.8510, -0.8353,  ..., -0.8902, -0.8902, -0.8902]],
 
         [[-0.7804, -0.7098, -0.6627,  ..., -0.8824, -0.9294, -0.9608],
          [-0.6627, -0.5922, -0.5608,  ..., -0.7961, -0.8431, -0.9059],
          [-0.5765, -0.5137, -0.4824,  ..., -0.7412, -0.7804, -0.8431],
          ...,
          [-0.7961, -0.7490, -0.7333,  ..., -0.7569, -0.7412, -0.7961],
          [-0.8667, -0.8510, -0.8196,  ..., -0.8039, -0.8118, -0.8588],
          [-0.8980, -0.9059, -0.8824,  ..., -0.8824, -0.8824, -0.8902]],
 
         [[-0.7647, -0.6392, -0.5608,  ..., -0.7804, -0.8745, -0.9373],
          [-0.5686, -0.4275,

In [7]:
len(datasets)

366

In [8]:
seed_gen=torch.Generator().manual_seed(42)

trainDS, validDS, testDS = random_split(datasets,
                                        [0.7, 0.1, 0.2],
                                        generator=seed_gen)

In [9]:
len(trainDS), len(validDS), len(testDS)

(257, 36, 73)

In [10]:
TRAIN_DL = DataLoader(trainDS, batch_size=9)
VALID_DL = DataLoader(validDS, batch_size=9)
TEST_DL = DataLoader(testDS, batch_size=9)


In [11]:
for a in TRAIN_DL:
    print(a)
    break

[tensor([[[[ 1.0000,  0.9608,  0.9451,  ...,  1.0000,  1.0000,  1.0000],
          [ 0.9686,  0.6235,  0.4118,  ...,  1.0000,  1.0000,  1.0000],
          [ 0.9686,  0.6078,  0.3569,  ...,  0.7020,  0.9686,  1.0000],
          ...,
          [ 0.9922,  0.5294, -0.4431,  ...,  0.7098,  1.0000,  1.0000],
          [ 0.9922,  0.9451,  0.3490,  ...,  0.6941,  1.0000,  0.9922],
          [ 0.9922,  1.0000,  0.9922,  ...,  0.9373,  1.0000,  0.9922]],

         [[ 1.0000,  0.9608,  0.9451,  ...,  1.0000,  1.0000,  1.0000],
          [ 0.9686,  0.6314,  0.4275,  ...,  1.0000,  1.0000,  1.0000],
          [ 0.9686,  0.6235,  0.3725,  ...,  0.7020,  0.9686,  1.0000],
          ...,
          [ 0.9922,  0.5216, -0.4510,  ...,  0.7176,  1.0000,  1.0000],
          [ 0.9843,  0.9373,  0.3412,  ...,  0.6471,  1.0000,  0.9922],
          [ 0.9922,  1.0000,  0.9922,  ...,  0.9294,  1.0000,  0.9922]],

         [[ 1.0000,  0.9529,  0.9373,  ...,  1.0000,  1.0000,  1.0000],
          [ 0.9686,  0.6863, 

In [12]:
device = 'cpu'

In [13]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [14]:
num_epochs = 20
learning_rate = 0.001
model = CNN(num_classes=3)  # Assuming you have 10 classes
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [15]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0

    for inputs, labels in TRAIN_DL:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_accuracy += (predicted == labels).sum().item() / labels.size(0)
        torch.save(model.state_dict(), 'albumart_model.pth')  # Add this line after training 


    model.eval()
    valid_loss = 0.0
    valid_accuracy = 0.0

    with torch.no_grad():
        for inputs, labels in VALID_DL:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            valid_accuracy += (predicted == labels).sum().item() / labels.size(0)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(TRAIN_DL):.4f}, Train Accuracy: {train_accuracy/len(TRAIN_DL):.4f}, Valid Loss: {valid_loss/len(VALID_DL):.4f}, Valid Accuracy: {valid_accuracy/len(VALID_DL):.4f}')


Epoch [1/20], Train Loss: 0.8613, Train Accuracy: 0.5073, Valid Loss: 0.9182, Valid Accuracy: 0.5278
Epoch [2/20], Train Loss: 0.8342, Train Accuracy: 0.4950, Valid Loss: 0.9157, Valid Accuracy: 0.3889
Epoch [3/20], Train Loss: 0.8144, Train Accuracy: 0.4958, Valid Loss: 0.8569, Valid Accuracy: 0.5556
Epoch [4/20], Train Loss: 0.7690, Train Accuracy: 0.5563, Valid Loss: 0.8598, Valid Accuracy: 0.5278
Epoch [5/20], Train Loss: 0.7381, Train Accuracy: 0.6023, Valid Loss: 0.8798, Valid Accuracy: 0.4722
Epoch [6/20], Train Loss: 0.6981, Train Accuracy: 0.6291, Valid Loss: 0.9316, Valid Accuracy: 0.5278
Epoch [7/20], Train Loss: 0.6467, Train Accuracy: 0.6789, Valid Loss: 0.9709, Valid Accuracy: 0.5556
Epoch [8/20], Train Loss: 0.5748, Train Accuracy: 0.7364, Valid Loss: 1.0764, Valid Accuracy: 0.6111
Epoch [9/20], Train Loss: 0.4952, Train Accuracy: 0.7862, Valid Loss: 1.1308, Valid Accuracy: 0.5556
Epoch [10/20], Train Loss: 0.4028, Train Accuracy: 0.8429, Valid Loss: 1.1917, Valid Accura

In [16]:
model.eval()
test_loss = 0.0
test_accuracy = 0.0

with torch.no_grad():
    for inputs, labels in TEST_DL:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        test_accuracy += (predicted == labels).sum().item() / labels.size(0)

print(f'Test Loss: {test_loss/len(TEST_DL):.4f}, Test Accuracy: {test_accuracy/len(TEST_DL):.4f}')

Test Loss: 2.1646, Test Accuracy: 0.5556
